# Script to generate EMG Forward Solution based onn MNE-python code

In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne.forward import make_forward_solution, compute_orient_prior
from pathlib import Path
import pymeshfix
import scipy
import pandas as pd
from scipy.spatial import ConvexHull, Delaunay
from mpl_toolkits.mplot3d import Axes3D
import pyvistaqt

%matplotlib qt

In [2]:
# Set conductivity and paths
conductivity = (2.67e-1, 4.06e-2, 2e-4,) # muscle, fat, skin
# Conductivities taken from at 100 Hz:  https://itis.swiss/virtual-population/tissue-properties/database/dielectric-properties/

# Does not actually require freesurfer installed, this is just the path to the directory
subjects_dir = Path('/Applications/freesurfer/7.3.2/subjects')
subject = "simp_arm"
bem_dir = subjects_dir / subject / "bem"

In [3]:
# Convert .obj files into MNE-named surfaces
coords, faces = mne.read_surface("/Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/Blender/Outer_skin.obj")
mne.write_surface(bem_dir / 'outer_skin.surf', coords, faces, overwrite=True)

coords, faces = mne.read_surface("/Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/Blender/Outer_fat.obj")
mne.write_surface(bem_dir / 'outer_skull.surf', coords, faces, overwrite=True)

coords, faces = mne.read_surface("/Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/Blender/Outer_muscle.obj")
mne.write_surface(bem_dir / 'inner_skull.surf', coords, faces, overwrite=True)

Overwriting existing file.
Overwriting existing file.
Overwriting existing file.


In [4]:
# If cleaning is required / if meshes have topological errors, can use pymeshfix

# coords, faces = mne.read_surface("/Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/Blender/Outer_skin_denseT.obj")
# mne.write_surface(bem_dir / 'inner_skull.surf', coords, faces, overwrite=True)

# coords, faces = pymeshfix.clean_from_arrays(coords, faces)

# coords = coords - coords.mean(axis=0)
# coords = coords / 1000  # Seems to be given in milimetres as, convert to meters

# mne.write_surface(bem_dir / 'inner_skull.surf', coords, faces, overwrite=True)

In [5]:
# Make the BEM model

# NOTE: You will need to comment out lines 630 to 631 in bem.py of mne-python's source code, namely the _check_surface_size(surf) function in _surfaces_to_bem().
# This is because the function checks for the size of the surface and throws an error if it is too small.  Clearly an arm is radially smaller than a head.

model = mne.make_bem_model(subject=subject, subjects_dir=subjects_dir, ico=None, conductivity=conductivity, verbose=None)

Creating the BEM geometry...
outer skin  CM is   0.00   0.00  85.17 mm
outer skull CM is   0.00   0.00  85.16 mm
inner skull CM is   0.00  -0.00  85.14 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    0.3 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    0.8 mm
Surfaces passed the basic topology checks.
Complete.



In [6]:
bem = mne.make_bem_solution(model,) # does not work with openmeeg, use MNE-Python's default BEM solver instead 

Three-layer model surfaces loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        outer skin  (7604) -> outer skin  (7604) ...
        outer skin  (7604) -> outer skull (7604) ...
        outer skin  (7604) -> inner skull (7604) ...
        outer skull (7604) -> outer skin  (7604) ...
        outer skull (7604) -> outer skull (7604) ...
        outer skull (7604) -> inner skull (7604) ...
        inner skull (7604) -> outer skin  (7604) ...
        inner skull (7604) -> outer skull (7604) ...
        inner skull (7604) -> inner skull (7604) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.


## Start from here if wanting to skip the BEM generation which takes the longest amount of time.

In [11]:
# mne.write_bem_solution("Data/bem_simp_arm", bem, overwrite=False, verbose=None)

bem = mne.read_bem_solution("Data/bem_simp_arm")

In [8]:
# Construct source space.  Adjust pos parameter to change the spacing between dipole sources.  Somewhere between 2mm and 5mm is sensible (~1,600 -> ~25,000 dipoles)
src = mne.setup_volume_source_space(subject=None, pos=2.0, mri=None, bem=bem, mindist=2.0, )

BEM              : <ConductorModel | BEM (3 layers) solver=mne>
grid                  : 2.0 mm
mindist               : 2.0 mm

Taking inner skull from <ConductorModel | BEM (3 layers) solver=mne>
Surface CM = (   0.0   -0.0   85.1) mm
Surface fits inside a sphere with radius   87.3 mm
Surface extent:
    x =  -21.6 ...   21.6 mm
    y =  -21.6 ...   21.6 mm
    z =    0.1 ...  169.8 mm
Grid extent:
    x =  -22.0 ...   22.0 mm
    y =  -22.0 ...   22.0 mm
    z =    2.0 ...  170.0 mm
44965 sources before omitting any.
44713 sources after omitting infeasible sources not within 0.0 - 87.3 mm.
Source spaces are in MRI coordinates.
Checking that the sources are inside the surface and at least    2.0 mm away (will take a few...)
Checking surface interior status for 44713 points...
    Found  3984/44713 points inside  an interior sphere of radius   19.7 mm
    Found     0/44713 points outside an exterior sphere of radius   87.3 mm
    Found 18749/40729 points outside using surface Qhull
    

In [9]:
# Check sources
src.plot(skull=False)

Using pyvistaqt 3d backend.


In [26]:
# Load electrode positions and move towards the skin surface

elec_pos_df = pd.read_csv("256leftarm_electrode_pos.csv", )

# Outer skin surface
pos, _ = mne.read_surface("/Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/Blender/Outer_skin.obj")
pos = pos/1000

# If accidentally used Blender's default export settings, will need to adjust the axes of the electrode positions (as this is easier) and the readjust everything later ["Y", "Z", "X"]
electrode_pos = elec_pos_df[["Y", "X", "Z"]].to_numpy()
# Then adjust translations for X and Y
electrode_pos[:, 0] = electrode_pos[:, 0] - electrode_pos[:,0].mean()
electrode_pos[:, 1] = electrode_pos[:, 1] - electrode_pos[:, 1].mean()
# Since electrode pos was mapped for left arm, swap x axis (where x-axis is along the electrodes) - Index 0 is correct as confirmed with the plot of source space and convex hull below.
electrode_pos[:, 0] = -electrode_pos[:, 0]

In [27]:
print(electrode_pos.max(axis=0))
print(electrode_pos.min(axis=0))
print(electrode_pos.mean(axis=0))
print(np.median(electrode_pos, axis=0))
print('src below')  # src['rr'] includes points not used, so need to adjust
print(src[0]['rr'].max(axis=0))
print(src[0]['rr'].min(axis=0))
print(src[0]['rr'].mean(axis=0))
print(np.median(src[0]['rr'], axis=0))
print('Outer skin below')
print(pos.max(axis=0))
print(pos.min(axis=0))
print(pos.mean(axis=0))
print(np.median(pos, axis=0))

elec_pos_df

[0.02468629 0.03548008 0.1575    ]
[-0.02964351 -0.03197114  0.        ]
[-9.54097912e-18  5.52943108e-18  7.71563691e-02]
[ 0.00075193 -0.00328633  0.07452097]
src below
[0.022 0.022 0.17 ]
[-0.022 -0.022  0.002]
[-5.75604899e-20 -1.07652005e-18  8.60000000e-02]
[0.    0.    0.086]
Outer skin below
[0.02886069 0.02886069 0.17      ]
[-0.02886069 -0.02886069  0.        ]
[3.28774320e-10 2.98264057e-10 8.51738242e-02]
[0.       0.       0.085175]


,X,Y,Z,RowColumn,Channel,Ch_num,Patch
0,0.027149,0.105739,2.596876e-06,R1C8,UNI 01,1,1
1,0.027056,0.105727,8.747513e-03,R2C8,UNI 02,2,1
2,0.027073,0.105645,1.751074e-02,R3C8,UNI 03,3,1
3,0.027503,0.105588,2.638230e-02,R4C8,UNI 04,4,1
4,0.020558,0.099517,9.829182e-07,R1C7,UNI 05,5,1
...,...,...,...,...,...,...,...
251,0.065396,0.098291,1.227922e-01,R8C7,UNI 60,60,8
252,0.051508,0.095327,1.574933e-01,R5C8,UNI 61,61,8
253,0.052548,0.096637,1.488152e-01,R6C8,UNI 62,62,8
254,0.053588,0.097948,1.401372e-01,R7C8,UNI 63,63,8


In [28]:
# Move electrodes to surface of skin

# Compute the convex hull
hull = ConvexHull(pos)

# Create a Delaunay triangulation of the convex hull vertices
tri = Delaunay(pos[hull.vertices])

# Shift electrode positions to the closest point on the convex hull surface

# Extract the vertices of the convex hull
hull_points = pos[hull.vertices]

# Function to find the closest point on a triangle
def closest_point_on_triangle(triangle, point):
    # Using Barycentric coordinates to find the closest point on the triangle
    A, B, C = triangle
    AB = B - A
    AC = C - A
    AP = point - A

    d1 = np.dot(AB, AP)
    d2 = np.dot(AC, AP)
    d3 = np.dot(AB, AB)
    d4 = np.dot(AC, AC)
    d5 = np.dot(AB, AC)

    denom = d3 * d4 - d5 * d5
    v = (d4 * d1 - d5 * d2) / denom
    w = (d3 * d2 - d5 * d1) / denom
    u = 1 - v - w

    if u >= 0 and v >= 0 and w >= 0:
        return u * A + v * B + w * C
    else:
        # If the point is outside the triangle, project it onto the closest edge
        def project_point_on_segment(P, Q, R):
            PQ = Q - P
            t = np.dot(R - P, PQ) / np.dot(PQ, PQ)
            t = np.clip(t, 0, 1)
            return P + t * PQ

        closest_points = [
            project_point_on_segment(A, B, point),
            project_point_on_segment(B, C, point),
            project_point_on_segment(C, A, point)
        ]
        distances = [np.linalg.norm(point - cp) for cp in closest_points]
        return closest_points[np.argmin(distances)]

# Function to find the closest point on the convex hull surface
def closest_point_on_hull(hull, point):
    closest_point = None
    min_distance = float('inf')
    for simplex in hull.simplices:
        triangle = hull.points[simplex]
        cp = closest_point_on_triangle(triangle, point)
        distance = np.linalg.norm(cp - point)
        if distance < min_distance:
            min_distance = distance
            closest_point = cp
    return closest_point

# Move all points in electrode_pos to the closest point on the convex hull surface
electrode_pos = np.array([closest_point_on_hull(hull, point) for point in electrode_pos])

In [29]:
# Plot the convex hull and the moved points
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Plot the convex hull
for simplex in hull.simplices:
    ax.plot(pos[simplex, 0], pos[simplex, 1], pos[simplex, 2], 'r-')
# Plot the moved points on the hull
ax.scatter(electrode_pos[:, 0], electrode_pos[:, 1], electrode_pos[:, 2], c=elec_pos_df["Patch"], marker='o')
# Plot the source space
ax.scatter(src[0]["rr"][src[0]["vertno"], 0], src[0]["rr"][src[0]["vertno"], 1], src[0]["rr"][src[0]["vertno"], 2], c='b', marker='o', alpha=0.5)
# Set labels
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
fig.show()

print('Want electrodes to go along x-axis, proximal to distal to go along z-axis, palmside in positive y direction')

Want electrodes to go along x-axis, proximal to distal to go along z-axis, palmside in positive y direction


In [30]:
# Save new electrode positions
# np.save("Data/256simparm_electrode_pos.npy", electrode_pos)

In [31]:
# Create info structure with EMG electrode channel names
ch_names = ['EMG ' + str(i) + '-' +str(j) for i, j in zip(elec_pos_df['Patch'],elec_pos_df['Ch_num'])]
info = mne.create_info(ch_names, sfreq=1000., ch_types='eeg')  # eeg type since MNE python does not like other types

# Set electrode positions with montage
montage = mne.channels.make_dig_montage(ch_pos={f'{ch}': loc for ch, loc in zip(ch_names, electrode_pos)} , coord_frame='head')
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: EMG 1-1, EMG 1-2, EMG 1-3, EMG 1-4, EMG 1-5, EMG 1-6, EMG 1-7, ...
 chs: 256 EEG
 custom_ref_applied: False
 dig: 259 items (3 Cardinal, 256 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 256
 projs: []
 sfreq: 1000.0 Hz
>

In [32]:
fwd = mne.make_forward_solution(info=info, trans=None, src=src, bem=bem, eeg=True)

Source space          : <SourceSpaces: [<discrete, n_used=23424>] MRI (surface RAS) coords, ~11.2 MB>
MRI -> head transform : identity
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 1 source spaces a total of 23424 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    1.000000 0.000000 0.000000       0.00 mm
    0.000000 1.000000 0.000000       0.00 mm
    0.000000 0.000000 1.000000       0.00 mm
    0.000000 0.000000 0.000000       1.00

Read 256 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Employing the head->MRI coordinate transform with the BEM model.
BEM model instance of ConductorModel is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface (will take a few...)
Checking surface interior status for 23424 points

In [35]:
# mne.write_forward_solution("Data/simp_arm_2mm-fwd.fif", fwd, overwrite=True)

fwd['sol']['data'].shape

Overwriting existing file.
    Write a source space...
    [done]
    1 source spaces written


(256, 70272)

Example of how it might look to generate fwd model with MNE-python's dipole fitting functions instead - Generated with ChatGPT

In [34]:
# # Define a 3D space representing the arm (can be replaced by a real anatomical model)
# # For simplicity, we simulate this as a unit cube.
# src = mne.setup_volume_source_space(subject=None, pos=10.0, mri=None, bem=None, mindist=5.0, sphere=(0, 0, 0, 0.5))

# # # Let's visualize the source space
# # fig = mne.viz.plot_bem(subject=None, brain_surfaces=None, src=src)
# # plt.title('Source Space (muscle sources for EMG)')
# # plt.show()

# src.plot(skull=False)

# # Simulate some arbitrary electrode positions (for simplicity, on the surface of the unit cube)
# # In reality, you would use the real positions of EMG electrodes.
# electrode_positions = np.array([
#     [0.5, 0, 0],  # Electrode 1
#     [0, 0.5, 0],  # Electrode 2
#     [0, 0, 0.5],  # Electrode 3
#     [-0.5, 0, 0], # Electrode 4
#     [0, -0.5, 0]  # Electrode 5
# ])

# # Create an info structure with EMG electrode channel names
# ch_names = ['EMG 1', 'EMG 2', 'EMG 3', 'EMG 4', 'EMG 5']
# info = mne.create_info(ch_names, sfreq=1000., ch_types='eeg')

# # Simulate a dipole at some location in the muscle (within the unit cube)
# dipole_location = np.array([0.2, 0.2, 0.2])  # Simulated muscle dipole location
# dipole_orientation = np.array([1, 0, 0])     # Orientation of dipole (direction of muscle contraction)

# # Define the dipole
# dipole = mne.Dipole(times=0, amplitude=np.array([[1.0, ]]), # Amplitude of each dipole 
#                     pos=dipole_location.reshape(1, 3), 
#                     ori=dipole_orientation.reshape(1, 3), 
#                     gof=1)

# montage = mne.channels.make_dig_montage(ch_pos={f'EMG {i}': loc for i, loc in enumerate(electrode_positions, 1)},
#                                         coord_frame='head')
# info.set_montage(montage)

# # Define the sphere model to represent the arm (simplified)
# # For this, we'll use a spherical volume conductor model
# radius = 0.1  # Example: radius of 10 cm (you can adjust this for arm size)
# sphere_model = mne.make_sphere_model(r0=(0., 0., 0.), head_radius=radius)

# # Compute the forward solution for the dipole (EMG potentials at electrodes)
# fwd_dipole = mne.make_forward_dipole(dipole, bem=sphere_model, info=info, trans=None, ) #bem=None

# # Visualize the dipole and electrode positions
# fig = mne.viz.plot_alignment(info, src=src, eeg=True)
# mne.viz.plot_dipole_locations(dipole, trans=None, mode='3d', subject=None, subjects_dir=None, ax=fig)
# plt.show()